In [5]:
import tensorflow as tf
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices('GPU'))


/device:GPU:0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
pip install -q yahoo_fin

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 4.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import seaborn as sns

In [4]:
!mkdir -p data results log
! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Todos:

 

1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [1]:
from runml import pipeline,findata

In [57]:
from importlib import reload
reload(pipeline)
reload(findata)

<module 'runml.findata' from '/Users/ahsank/src/runml/runml/findata.py'>

In [15]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE=False
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MAXR', 'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM', 'RBLX',
           'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM', 'STNE','SWAV',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
# # RateReturnOnlyExperimental
# pipeline.PenaltyTrading.threshold=0.2

lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200), pipeline.Adj()]))
df = pipeline.runModelCombinedVola(tickers, 'ipostest4a', mod, True, loss=lossfn)

Epoch 1/200
728/728 [==============================] - ETA: 0s - loss: 0.0050 - mean_absolute_error: 0.0726
Epoch 1: val_loss improved from inf to 0.00412, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [==============================] - 25s 32ms/step - loss: 0.0050 - mean_absolute_error: 0.0726 - val_loss: 0.0041 - val_mean_absolute_error: 0.0660
Epoch 2/200
728/728 [==============================] - ETA: 0s - loss: 0.0043 - mean_absolute_error: 0.0677
Epoch 2: val_loss did not improve from 0.00412
728/728 [==============================] - 21s 29ms/step - loss: 0.0043 - mean_absolute_error: 0.0677 - val_loss: 0.0043 - val_mean_absolute_error: 0.0654
Epoch 3/200
728/728 [==============================] - ETA: 0s - loss: 0.0040 - mean_absolute_error: 0.0649
Epoch 3: val_loss improved from 0.00412 to 0.00375, saving model to results/ipostest4a-adjclose-sh-1-sc-1-s

728/728 [==============================] - 20s 28ms/step - loss: 0.0026 - mean_absolute_error: 0.0522 - val_loss: 0.0027 - val_mean_absolute_error: 0.0533
Epoch 22/200
727/728 [============================>.] - ETA: 0s - loss: 0.0025 - mean_absolute_error: 0.0515
Epoch 22: val_loss improved from 0.00271 to 0.00256, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [==============================] - 20s 28ms/step - loss: 0.0025 - mean_absolute_error: 0.0515 - val_loss: 0.0026 - val_mean_absolute_error: 0.0512
Epoch 23/200
728/728 [==============================] - ETA: 0s - loss: 0.0024 - mean_absolute_error: 0.0504
Epoch 23: val_loss improved from 0.00256 to 0.00254, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [=============================

727/728 [============================>.] - ETA: 0s - loss: 8.0021e-04 - mean_absolute_error: 0.0302
Epoch 41: val_loss improved from 0.00105 to 0.00104, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [==============================] - 21s 29ms/step - loss: 8.0056e-04 - mean_absolute_error: 0.0303 - val_loss: 0.0010 - val_mean_absolute_error: 0.0348
Epoch 42/200
726/728 [============================>.] - ETA: 0s - loss: 7.6704e-04 - mean_absolute_error: 0.0297
Epoch 42: val_loss improved from 0.00104 to 0.00091, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [==============================] - 21s 29ms/step - loss: 7.6656e-04 - mean_absolute_error: 0.0297 - val_loss: 9.1165e-04 - val_mean_absolute_error: 0.0327
Epoch 43/200
727/728 [=========

Epoch 60/200
727/728 [============================>.] - ETA: 0s - loss: 3.7361e-04 - mean_absolute_error: 0.0206
Epoch 60: val_loss improved from 0.00058 to 0.00058, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [==============================] - 20s 28ms/step - loss: 3.7338e-04 - mean_absolute_error: 0.0206 - val_loss: 5.7599e-04 - val_mean_absolute_error: 0.0257
Epoch 61/200
727/728 [============================>.] - ETA: 0s - loss: 3.7391e-04 - mean_absolute_error: 0.0206
Epoch 61: val_loss improved from 0.00058 to 0.00049, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [==============================] - 21s 28ms/step - loss: 3.7401e-04 - mean_absolute_error: 0.0206 - val_loss: 4.9335e-04 - val_mean_absolute_error: 0.0233
Epoch 62/200
7

727/728 [============================>.] - ETA: 0s - loss: 2.6348e-04 - mean_absolute_error: 0.0170
Epoch 83: val_loss did not improve from 0.00043
728/728 [==============================] - 20s 28ms/step - loss: 2.6353e-04 - mean_absolute_error: 0.0170 - val_loss: 5.0207e-04 - val_mean_absolute_error: 0.0237
Epoch 84/200
726/728 [============================>.] - ETA: 0s - loss: 2.4452e-04 - mean_absolute_error: 0.0164
Epoch 84: val_loss did not improve from 0.00043
728/728 [==============================] - 20s 28ms/step - loss: 2.4523e-04 - mean_absolute_error: 0.0164 - val_loss: 5.0950e-04 - val_mean_absolute_error: 0.0238
Epoch 85/200
728/728 [==============================] - ETA: 0s - loss: 2.5049e-04 - mean_absolute_error: 0.0167
Epoch 85: val_loss did not improve from 0.00043
728/728 [==============================] - 21s 29ms/step - loss: 2.5049e-04 - mean_absolute_error: 0.0167 - val_loss: 4.4183e-04 - val_mean_absolute_error: 0.0223
Epoch 86/200
726/728 [===================

Epoch 108/200
727/728 [============================>.] - ETA: 0s - loss: 1.8686e-04 - mean_absolute_error: 0.0142
Epoch 108: val_loss did not improve from 0.00040
728/728 [==============================] - 21s 28ms/step - loss: 1.8686e-04 - mean_absolute_error: 0.0142 - val_loss: 4.3733e-04 - val_mean_absolute_error: 0.0217
Epoch 109/200
726/728 [============================>.] - ETA: 0s - loss: 1.8409e-04 - mean_absolute_error: 0.0143
Epoch 109: val_loss improved from 0.00040 to 0.00040, saving model to results/ipostest4a-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
728/728 [==============================] - 20s 28ms/step - loss: 1.8405e-04 - mean_absolute_error: 0.0143 - val_loss: 3.9696e-04 - val_mean_absolute_error: 0.0216
Epoch 110/200
728/728 [==============================] - ETA: 0s - loss: 1.7954e-04 - mean_absolute_error: 0.0141
Epoch 110: val_loss did not improve from 0.00040
728/728 [=======

In [27]:

#testdf = res['MSFT'].data
adddf = res['MSFT'].final_df
# #pd.set_option('display.float_format', '{:.2f}'.format)
# df = df.drop(columns=['ma-adjclose-200', 'vwap', 'month', 'mday', 'open', 'volume', 'buy_profit', 'sell_profit'])
# for col in list(df):
#     if df[col].dtype == np.float64:
#         df[col] = df[col].round(2)

# print ("df = pd.DataFrame( %s )" % (str(df.reset_index(drop=True).to_dict())))
#testdf['test_df']
adddf.head(20)


,high,pred_high,true_high,adjclose,true_adjclose,buy_profit,sell_profit
2019-07-03,132.689362,138.613083,135.650177,132.409363,135.039047,2.629684,0.00000
2019-07-05,132.294022,138.489929,136.486755,132.024017,136.146759,4.122742,0.00000
2019-07-12,134.026443,138.954788,136.486755,133.796432,131.869904,-1.926529,0.00000
2019-07-19,135.650177,139.937119,136.486755,131.600174,132.650162,1.049988,0.00000
2019-07-22,134.103699,140.345413,136.486755,133.343689,130.800720,-2.542969,0.00000
2019-07-26,136.486755,144.330475,136.328186,136.146759,131.564865,-4.581894,0.00000
2019-08-09,134.320160,137.379501,134.835693,132.650162,133.236832,0.586670,0.00000
2019-08-13,133.707474,138.547577,134.835693,133.507477,133.014557,-0.492920,0.00000
2019-08-15,130.097015,138.856415,135.683411,129.197006,134.435287,5.238281,0.00000
2019-08-16,131.894867,139.074966,135.683411,131.564865,132.908249,1.343384,0.00000


In [59]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
Ticker,Col,,,,,,,,,,
AI,adjclose,RROnly,13.370372,0.714286,66.49,364.80,431.29,22.75,48.689999,1.14,6.892231
ABNB,adjclose,RROnly,87.175750,0.904762,603.22,627.28,1230.50,114.25,109.879997,-0.04,6.666667
AMZN,adjclose,RROnly,84.672864,0.856383,760.69,718.45,1479.14,99.92,105.449997,0.06,6.170213
MSFT,adjclose,RROnly,121.268275,0.824468,1190.31,664.21,1854.52,282.83,270.119995,-0.04,3.856383
AI,high,RROnly,14.525238,0.266667,0.00,101.93,101.93,22.75,36.220001,0.59,1.423729
AMBA,low,RROnly,38.026026,0.500000,742.49,0.00,742.49,71.22,61.779999,-0.13,1.153846
APP,low,RROnly,10.598792,0.375000,130.65,0.00,130.65,15.96,12.770000,-0.20,0.625000
AMZN,high,RROnly,87.579336,0.324468,0.00,352.09,352.09,99.92,104.879997,0.05,0.000000
MSFT,low,RROnly,118.024078,0.234043,508.32,0.00,508.32,282.83,263.869995,-0.07,0.000000


In [10]:
data = pipeline.fetch_data('MSFT')
data = data.tail(1000)


,open,high,low,close,adjclose,volume,ticker
2019-04-12,120.639999,120.980003,120.370003,120.949997,116.076286,19745100,MSFT
2019-04-15,120.940002,121.580002,120.570000,121.050003,116.172279,15792600,MSFT
2019-04-16,121.639999,121.650002,120.099998,120.769997,115.903542,14071800,MSFT
2019-04-17,121.239998,121.849998,120.540001,121.769997,116.863243,19300900,MSFT
2019-04-18,122.190002,123.519997,121.300003,123.370003,118.398781,27991000,MSFT
...,...,...,...,...,...,...,...
2023-03-27,280.500000,281.459991,275.519989,276.380005,276.380005,26840200,MSFT
2023-03-28,275.790009,276.140015,272.049988,275.230011,275.230011,21878600,MSFT
2023-03-29,278.959991,281.140015,278.410004,280.510010,280.510010,25087000,MSFT
2023-03-30,284.230011,284.459991,281.480011,284.049988,284.049988,25053400,MSFT


In [23]:

data['adjhigh'] = data.high/data.close*data.adjclose
data['adjlow'] = data.low/data.close*data.adjclose
data['future'] = data.adjclose.shift(-15)
data['min']  = data.adjlow.rolling(15).min().shift(-15)
data.tail(50).dropna()

,open,high,low,close,adjclose,volume,ticker,adjhigh,adjlow,future,min
2023-01-20,234.860001,240.740005,234.509995,240.220001,239.619827,35389800,MSFT,240.138532,233.924087,262.442657,230.323103
2023-01-23,241.100006,245.169998,239.649994,242.580002,241.973923,31934000,MSFT,244.557448,239.051235,270.642120,230.323103
2023-01-24,242.500000,243.949997,240.440002,242.039993,241.435272,40234400,MSFT,243.340504,239.839279,271.490021,230.323103
2023-01-25,234.479996,243.300003,230.899994,240.610001,240.008850,66526600,MSFT,242.692132,230.323103,269.320007,241.395377
2023-01-26,243.649994,248.309998,242.000000,248.000000,247.380386,33454500,MSFT,247.689609,241.395377,262.149994,241.594872
2023-01-27,248.990005,249.830002,246.830002,248.160004,247.539993,26498900,MSFT,249.205819,246.213314,258.059998,241.594872
2023-01-30,244.509995,245.600006,242.199997,242.710007,242.103607,25867400,MSFT,244.986386,241.594872,252.669998,242.342999
2023-01-31,243.449997,247.949997,242.949997,247.809998,247.190857,26541100,MSFT,247.330507,242.342999,251.509995,244.856705
2023-02-01,248.000000,255.179993,245.470001,252.750000,252.118515,31259900,MSFT,254.542436,244.856705,254.770004,250.339996
2023-02-02,258.820007,264.690002,257.250000,264.600006,263.938904,39940400,MSFT,264.028675,256.607262,249.220001,248.100006


In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV', 
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE', 
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE', 
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [34]:
import pandas as pd
df = pd.DataFrame({'Date':['10/2/2011', '11/2/2011', '12/2/2011', '13/2/2011'],
                    'Event':['Music', 'Poetry', 'Theatre', 'Comedy'],
                    'Cost':[10000, 5000, 15000, 2000]})
 
# Print the dataframe
print(df)

df['Discounted_Price'] = df.apply(lambda row: row.Cost -
                                  (row.Cost * 0.1), axis = 1)
 
# Print the DataFrame after addition
# of new column
print(df)

        Date    Event   Cost
0  10/2/2011    Music  10000
1  11/2/2011   Poetry   5000
2  12/2/2011  Theatre  15000
3  13/2/2011   Comedy   2000
        Date    Event   Cost  Discounted_Price
0  10/2/2011    Music  10000            9000.0
1  11/2/2011   Poetry   5000            4500.0
2  12/2/2011  Theatre  15000           13500.0
3  13/2/2011   Comedy   2000            1800.0


In [73]:
from pandas import Timestamp

df = pd.DataFrame({
    'adjlow': {
        0: 132.91, 1: 130.07, 2: 128.26, 3: 131.4, 4: 127.77, 5: 131.67, 6: 131.85, 7: 131.82, 8: 135.34, 9: 133.57,},
    'high': {
        0: 139.13, 1: 140.49, 2: 135.68, 3: 139.38, 4: 134.58,
        5: 139.2,
        6: 139.75,
        7: 137.52,
        8: 142.37,
        9: 141.65,
        },
    'low': {
        0: 138.01,
        1: 135.08,
        2: 133.21,
        3: 136.46,
        4: 132.25,
        5: 136.29,
        6: 136.46,
        7: 136.43,
        8: 140.07,
        9: 138.25,
        },
    'close': {
        0: 138.9,
        1: 136.27,
        2: 134.69,
        3: 137.71,
        4: 133.68,
        5: 137.78,
        6: 137.52,
        7: 137.39,
        8: 141.07,
        9: 139.44,
        },
    'adjclose': {
        0: 133.8,
        1: 131.26,
        2: 129.74,
        3: 132.65,
        4: 129.2,
        5: 133.16,
        6: 132.91,
        7: 132.78,
        8: 136.34,
        9: 134.76,
        },
    'ticker': {
        0: 'MSFT',
        1: 'MSFT',
        2: 'MSFT',
        3: 'MSFT',
        4: 'MSFT',
        5: 'MSFT',
        6: 'MSFT',
        7: 'MSFT',
        8: 'MSFT',
        9: 'MSFT',
        },
    'date': {
        0: Timestamp('2019-07-12 00:00:00'),
        1: Timestamp('2019-07-31 00:00:00'),
        2: Timestamp('2019-08-06 00:00:00'),
        3: Timestamp('2019-08-09 00:00:00'),
        4: Timestamp('2019-08-15 00:00:00'),
        5: Timestamp('2019-08-22 00:00:00'),
        6: Timestamp('2019-09-09 00:00:00'),
        7: Timestamp('2019-09-17 00:00:00'),
        8: Timestamp('2019-09-19 00:00:00'),
        9: Timestamp('2019-09-20 00:00:00'),
        },
    'pred_adjclose': {
        0: 134.43833923339844,
        1: 138.68714904785156,
        2: 137.3265380859375,
        3: 135.59080505371094,
        4: 135.3883514404297,
        5: 137.06692504882812,
        6: 136.4296417236328,
        7: 135.31634521484375,
        8: 136.20584106445312,
        9: 137.16757202148438,
        },
    'true_adjclose': {
        0: 131.87,
        1: 134.14,
        2: 131.19,
        3: 133.24,
        4: 134.44,
        5: 132.71,
        6: 134.37,
        7: 131.12,
        8: 134.44,
        9: 135.0,
        },
    })
df = df.set_index('date')
df

,adjlow,high,low,close,adjclose,ticker,pred_adjclose,true_adjclose
date,,,,,,,,
2019-07-12,132.91,139.13,138.01,138.90,133.80,MSFT,134.44,131.87
2019-07-31,130.07,140.49,135.08,136.27,131.26,MSFT,138.69,134.14
2019-08-06,128.26,135.68,133.21,134.69,129.74,MSFT,137.33,131.19
2019-08-09,131.40,139.38,136.46,137.71,132.65,MSFT,135.59,133.24
2019-08-15,127.77,134.58,132.25,133.68,129.20,MSFT,135.39,134.44
2019-08-22,131.67,139.20,136.29,137.78,133.16,MSFT,137.07,132.71
2019-09-09,131.85,139.75,136.46,137.52,132.91,MSFT,136.43,134.37
2019-09-17,131.82,137.52,136.43,137.39,132.78,MSFT,135.32,131.12
2019-09-19,135.34,142.37,140.07,141.07,136.34,MSFT,136.21,134.44


In [79]:
class NormalTrading:
    buy_profit  = lambda row : row.true_adjclose - row.adjclose if row.pred_adjclose > row.adjclose else 0
    sell_profit = lambda row: row.adjclose - row.true_adjclose if row.pred_adjclose < row.adjclose else 0


In [80]:

trade = NormalTrading
df["buy_profit"] = list(df.apply(trade.buy_profit, axis=1)
                                    # since we don't have profit for last sequence, add 0's
                                    )
df

,adjlow,high,low,close,adjclose,ticker,pred_adjclose,true_adjclose,buy_profit
date,,,,,,,,,
2019-07-12,132.91,139.13,138.01,138.90,133.80,MSFT,134.44,131.87,-1.93
2019-07-31,130.07,140.49,135.08,136.27,131.26,MSFT,138.69,134.14,2.88
2019-08-06,128.26,135.68,133.21,134.69,129.74,MSFT,137.33,131.19,1.45
2019-08-09,131.40,139.38,136.46,137.71,132.65,MSFT,135.59,133.24,0.59
2019-08-15,127.77,134.58,132.25,133.68,129.20,MSFT,135.39,134.44,5.24
2019-08-22,131.67,139.20,136.29,137.78,133.16,MSFT,137.07,132.71,-0.45
2019-09-09,131.85,139.75,136.46,137.52,132.91,MSFT,136.43,134.37,1.46
2019-09-17,131.82,137.52,136.43,137.39,132.78,MSFT,135.32,131.12,-1.66
2019-09-19,135.34,142.37,140.07,141.07,136.34,MSFT,136.21,134.44,0.00
